# Chains in SK

## Outline

* Sequential Chains with kernel.run_async


In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

In [3]:
#!pip install pandas

In [4]:
import pandas as pd
df = pd.read_csv('Data.csv')

In [5]:
df.head()

,Product,Review
0,Queen Size Sheet Set,I ordered a king size set. My only criticism w...
1,Waterproof Phone Pouch,"I loved the waterproof sac, although the openi..."
2,Luxury Air Mattress,This mattress had a small hole in the top of i...
3,Pillows Insert,This is the best throw pillow fillers on Amazo...
4,Milk Frother Handheld\n,I loved this product. But they only seem to l...


In [6]:
df.Review[5]

"Je trouve le goût médiocre. La mousse ne tient pas, c'est bizarre. J'achète les mêmes dans le commerce et le goût est bien meilleur...\nVieux lot ou contrefaçon !?"

## LLM Chain

In [7]:
import semantic_kernel as sk
import os
import logging
from semantic_kernel.connectors.ai.open_ai import OpenAIChatCompletion
# Demonstrate some basic logging to debug the chain
logging.basicConfig(level=logging.DEBUG)
logger = logging.getLogger('__name__')
kernel=sk.Kernel(log=logger)
api_key = os.environ['OPENAI_API_KEY']
kernel.add_chat_service(
        "chat-gpt", OpenAIChatCompletion("gpt-3.5-turbo", api_key)
)

In [8]:
prompt = "What is the best name to describe \
    a company that makes {{ $input }} ?"


In [9]:
# Create a semantic function based on prompt to name a company based on product it makes. 
# Make it creative by setting a high temperature for the LLM
productnamer = kernel.create_semantic_function(prompt, temperature=0.9)

DEBUG:__name__:Extracting blocks from template: What is the best name to describe     a company that makes {{ $input }} ?


In [10]:
product = "Queen Size Sheet Set"
answer = await kernel.run_async(productnamer, input_str=product)
print(answer)

DEBUG:__name__:Rendering string template: What is the best name to describe     a company that makes {{ $input }} ?
DEBUG:__name__:Extracting blocks from template: What is the best name to describe     a company that makes {{ $input }} ?
DEBUG:__name__:Rendering list of 3 blocks
DEBUG:__name__:Rendered prompt: What is the best name to describe     a company that makes Queen Size Sheet Set ?
DEBUG:openai:message='Request to OpenAI API' method=post path=https://api.openai.com/v1/chat/completions
DEBUG:openai:api_version=None data='{"model": "gpt-3.5-turbo", "messages": [{"role": "user", "content": "What is the best name to describe     a company that makes Queen Size Sheet Set ?"}], "temperature": 0.9, "top_p": 1.0, "presence_penalty": 0.0, "frequency_penalty": 0.0, "max_tokens": 256, "n": 1, "stream": false}' message='Post details'
INFO:openai:message='OpenAI API response' path=https://api.openai.com/v1/chat/completions processing_ms=382 request_id=53d13fc905105cee2d627d75b181a679 respo

Royal Comfort Beddings


## SimpleSequentialChain

In [12]:
# Now lets chain two skills together sequerntially with output of first passed as input to second prompt
# prompt template 1
first_prompt = "What is the best name to describe \
    a company that makes {{ $input }}?"

# Semantic Function 1
func_one = kernel.create_semantic_function(first_prompt, temperature=0.9)

DEBUG:__name__:Extracting blocks from template: What is the best name to describe     a company that makes {{ $input }}?


In [13]:

# prompt template 2
second_prompt = "Write a 20 words description for the following \
    company:{{ $input }}"

# chain 2
func_two = kernel.create_semantic_function(second_prompt, temperature=0.9)

DEBUG:__name__:Extracting blocks from template: Write a 20 words description for the following     company:{{ $input }}


In [14]:
answer = await kernel.run_async(func_one, func_two, input_str="Queen Size Sheet Set")
print(answer)


DEBUG:__name__:Rendering string template: What is the best name to describe     a company that makes {{ $input }}?
DEBUG:__name__:Extracting blocks from template: What is the best name to describe     a company that makes {{ $input }}?
DEBUG:__name__:Rendering list of 3 blocks
DEBUG:__name__:Rendered prompt: What is the best name to describe     a company that makes Queen Size Sheet Set?
DEBUG:openai:message='Request to OpenAI API' method=post path=https://api.openai.com/v1/chat/completions
DEBUG:openai:api_version=None data='{"model": "gpt-3.5-turbo", "messages": [{"role": "user", "content": "What is the best name to describe     a company that makes Queen Size Sheet Set?"}], "temperature": 0.9, "top_p": 1.0, "presence_penalty": 0.0, "frequency_penalty": 0.0, "max_tokens": 256, "n": 1, "stream": false}' message='Post details'
INFO:openai:message='OpenAI API response' path=https://api.openai.com/v1/chat/completions processing_ms=576 request_id=d463fa4449e8338ab4f6e3b9b1999210 response_

Royal Comfort Beddings is a luxury bedding company providing exquisite and comfortable bedding sets for a perfect night's sleep.


## Sequential Chain

Next we will look at more flexible ways to chain together semantic functions beyond a sequential pipeline. Here you need to save output of a step into context variables using Semantic Kernel's **Native Functions** which are registered as semantic functions. Currently you need to call these native semantic functions after an LLM function to store output to a specific context variable so it can be used in latter LLM functions. 

In [15]:
from semantic_kernel.skill_definition import sk_function
from semantic_kernel.skill_definition import sk_function, sk_function_context_parameter
from semantic_kernel import SKContext

class CopyContext:
    """
    Description: Copies context to do chaining
    """

    @sk_function(
        description="Copies input to another context variable"
    )
    @sk_function_context_parameter(name="English_Review", description="Review in English.")
    def EnglishReview(self, context: SKContext) -> str:
        context["English_Review"] = context["input"]
        return context["input"]


    @sk_function(
        description="Copies input to another context variable"
    )
    @sk_function_context_parameter(name="summary", description="Review in English.")
    def Summary(self, context: SKContext) -> str:
        context["summary"] = context["input"]
        return context["input"]
    
    @sk_function(
        description="Copies input to another context variable"
    )
    @sk_function_context_parameter(name="language", description="Review in English.")
    def Language(self, context: SKContext) -> str:
        context["language"] = context["input"]
        return context["input"]
    
    

In [16]:
copy_context_skill = kernel.import_skill(CopyContext())
v_englishreview = copy_context_skill["EnglishReview"]
v_language = copy_context_skill["Language"]
v_summary = copy_context_skill["Summary"]

DEBUG:__name__:Importing skill _GLOBAL_FUNCTIONS_ into the global namespace
DEBUG:__name__:Methods imported: 3


In [18]:
review=df.Review[5]
context_variables = sk.ContextVariables()
context_variables["review"] = review

In [19]:
# prompt template 1: translate to english
first_prompt = """Translate the following review to english:
    {{ $review }}"""

# chain 1: input= Review and output= English_Review
# Semantic Function 1
func_one = kernel.create_semantic_function(first_prompt, temperature=0.9)


DEBUG:__name__:Extracting blocks from template: Translate the following review to english:
    {{ $review }}


In [20]:
second_prompt = """Can you summarize the following review in 1 sentence:
    {{ $English_Review }}"""

# chain 2: input= English_Review and output= summary
func_two = kernel.create_semantic_function(second_prompt, temperature=0.9)

DEBUG:__name__:Extracting blocks from template: Can you summarize the following review in 1 sentence:
    {{ $English_Review }}


In [22]:
# Lets try to first chain two functions before trying a more complex chain
# Chain looks like this: func_one-> store output in context -> func_two
answer = await kernel.run_async(func_one, v_englishreview,func_two, input_vars=context_variables)
print(answer)

DEBUG:__name__:Rendering string template: Translate the following review to english:
    {{ $review }}
DEBUG:__name__:Extracting blocks from template: Translate the following review to english:
    {{ $review }}
DEBUG:__name__:Rendering list of 2 blocks
DEBUG:__name__:Rendered prompt: Translate the following review to english:
    Je trouve le goût médiocre. La mousse ne tient pas, c'est bizarre. J'achète les mêmes dans le commerce et le goût est bien meilleur...
Vieux lot ou contrefaçon !?
DEBUG:openai:message='Request to OpenAI API' method=post path=https://api.openai.com/v1/chat/completions
DEBUG:openai:api_version=None data='{"model": "gpt-3.5-turbo", "messages": [{"role": "user", "content": "Translate the following review to english:\\n    Je trouve le go\\u00fbt m\\u00e9diocre. La mousse ne tient pas, c\'est bizarre. J\'ach\\u00e8te les m\\u00eames dans le commerce et le go\\u00fbt est bien meilleur...\\nVieux lot ou contrefa\\u00e7on !?"}], "temperature": 0.9, "top_p": 1.0, "pre

The reviewer finds the taste of the product mediocre and suspects that it may be an old batch or counterfeit since the foam doesn't hold and the taste is not as good as the ones bought in stores.


In [23]:
# Now lets chain more functions together in more flexible manner
# prompt template 3: translate to english
third_prompt = "What language is the following review:\n\n{{ $review }}"

# chain 3: input= Review and output= language
func_three = kernel.create_semantic_function(third_prompt, temperature=0.9)

DEBUG:__name__:Extracting blocks from template: What language is the following review:

{{ $review }}


In [24]:
# prompt template 4: follow up message
fourth_prompt = """Write a follow up response to the following 
    summary in the specified language:
    Summary: {{ $summary }}\n\nLanguage: {{ $language }}"""

# chain 4: input= summary, language and output= followup_message
func_four = kernel.create_semantic_function(fourth_prompt, temperature=0.9)

DEBUG:__name__:Extracting blocks from template: Write a follow up response to the following 
    summary in the specified language:
    Summary: {{ $summary }}

Language: {{ $language }}


Now call a more elaborate chain where output from a step is used as input not just in next step
Chain looks like this:
<pre>
                                         +-----------------------------
                                         |                            |
                                         |                            V
 (Input)Review ----> func_one   --->  func_two  func_three  ----->  func_four
          |                                        ^
          |                                        |
          +-----------------------------------------
</pre>
   
Notice that after each function , we have to call a native function (like v_englighreview, v_summary, v_language) to store output in context so it can be referenced in a subsequent step. In kernel.run_async you can specify a list of functions (semantic or native function) to call which it does in sequence. 

In [31]:
answer = await kernel.run_async(func_one, v_englishreview,func_two, v_summary, func_three, v_language, func_four, input_vars=context_variables)

print(answer)

DEBUG:__name__:Rendering string template: Translate the following review to english:
    {{ $review }}
DEBUG:__name__:Extracting blocks from template: Translate the following review to english:
    {{ $review }}
DEBUG:__name__:Rendering list of 2 blocks
DEBUG:__name__:Rendered prompt: Translate the following review to english:
    Je trouve le goût médiocre. La mousse ne tient pas, c'est bizarre. J'achète les mêmes dans le commerce et le goût est bien meilleur...
Vieux lot ou contrefaçon !?
DEBUG:openai:message='Request to OpenAI API' method=post path=https://api.openai.com/v1/chat/completions
DEBUG:openai:api_version=None data='{"model": "gpt-3.5-turbo", "messages": [{"role": "user", "content": "Translate the following review to english:\\n    Je trouve le go\\u00fbt m\\u00e9diocre. La mousse ne tient pas, c\'est bizarre. J\'ach\\u00e8te les m\\u00eames dans le commerce et le go\\u00fbt est bien meilleur...\\nVieux lot ou contrefa\\u00e7on !?"}], "temperature": 0.9, "top_p": 1.0, "pre

Réponse:
Nous vous remercions d'avoir partagé votre avis sur notre produit. Nous sommes désolés d'apprendre que le goût ne vous a pas convaincu et que la mousse n'était pas constante. Nous tenons à vous assurer que nous prenons cela au sérieux et que nous enquêterons sur la possibilité d'un lot ancien ou d'une version contrefaite. Votre satisfaction est notre priorité et nous ferons tout notre possible pour rectifier cette situation. Si vous avez d'autres préoccupations ou des questions supplémentaires, n'hésitez pas à nous contacter. Merci encore pour votre feedback constructif.
